# Soccer Mobile
<!-- Introduction of the project -->

In [ ]:
from global_navigation import *
from motion_control import *

## Vision


In [ ]:
# ...
gmap = None
pBall = Pos() #
pGoal = Pos() #

## Global Navigation
Plan a path from the start to the goal.
Return: Waypoints

In [ ]:
planner =  PathPlanner(gmap)

## Local Navigation


## Filtering
tell where Thymio is.

## Motion Control
Actuator.

In [ ]:
controller = MotionController()
control_cycle = 10 # ms = 100Hz

In [ ]:
get_ball = False
# 1. get the ball
planner.set_goal = pBall
planner.set_start = None # Pos of Thymio [Filtering]
waypoints = planner.plan()
while not get_ball:
    if controller.obs_forward():
        pass #   [Local Navigation]
        #          return to waypoints to follow
    else:
        waypoints = controller.go_pos(waypoints) # follow the waypoints 
    pass

# 2. Push the ball to the goal
finished = False
planner.set_goal = pGoal
planner.set_start = None # Pos of Thymio [Filtering]
waypoints = planner.plan()
while not finished:
    if controller.obs_forward():
        pass #   [Local Navigation]
        #          return to waypoints to follow
    else:
        waypoints = controller.go_pos(waypoints) # follow the waypoints 
    pass

# Serial Control vs TDMClient

In [1]:
from Thymio import Thymio
import os
import sys

In [4]:
!pip install pyserial

In [5]:
import sys
import glob
import serial


def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result


if __name__ == '__main__':
    print(serial_ports())

[]


In [6]:
sys.setrecursionlimit(3000)
print(os.getenv('COM4'))

None


In [7]:
th = Thymio.serial(port=os.getenv('COM6'), refreshing_rate=0.1)

IndexError: list index out of range